# ITD and ILD

In [ ]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt
import soundfile as sf
from IPython.display import display, Audio

### Interaural Time Difference (ITD)

$R_{LR}(\tau) = \sum_{t} x_{L}(t) \cdot x_{R}(t + \tau)$

$ITD = \frac{\text{arg max}_{\tau}\, R_{LR}(\tau)}{f_s}$

In [ ]:
def compute_itd(y, srate):
    left = y[:, 0]  # left signal
    right = y[:, 1]  # right signal
  
    # cross-correlation
    cross_corr = signal.correlate(left, right, mode='full')
    
    # calculate lag
    lags = np.arange(-len(left) + 1, len(right))
    lag = lags[np.argmax(cross_corr)]
    
    # convert lag to time
    itd = lag / srate

    return itd

### Interaural Level Difference (ILD)

$RMS_L = \sqrt{\frac{1}{N} \sum_{t=1}^{N}{x_L(t)^2}}$

$RMS_R = \sqrt{\frac{1}{N} \sum_{t=1}^{N}{x_R(t)^2}}$

$ILD = 20 \cdot \log_{10} (\frac{RMS_L}{RMS_R})$

In [ ]:
def compute_ild(y):
    left = y[:, 0]  # left signal
    right = y[:, 1]  # right signal

    # compute the rms across the entire channel
    rms_left = np.sqrt(np.mean(left ** 2))
    rms_right = np.sqrt(np.mean(right ** 2))

    # small error in case of divison by zero
    eps = 1e-12

    ild = 20 * np.log10(rms_left / (rms_right + eps))

    return ild

## Evaluate

In [ ]:
DATE =  datetime.now().strftime("%Y-%m-%d")
MODEL = 'spleeter_test'
EVAL_DIR = "../data/eval/itd_ild/"

In [ ]:
STEMS = ["drums", "bass", "other", "vocals"]

### Stereo Data

In [ ]:
DATASET = 'stereo'

# set input and output directories
REFERENCE_DIR = f"../data/musdb18hq/test/"
ESTIMATE_DIR = f"../data/output/{MODEL}/{DATASET}/test/"

In [ ]:
# create the output directory if it does not already exist
print("Creating evaluation directory, if it does not already exist...")
os.makedirs(EVAL_DIR, exist_ok=True)

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
song_list = [f for f in os.listdir(REFERENCE_DIR) if os.path.isdir(os.path.join(REFERENCE_DIR, f))]
print(f"There are {len(song_list)} files in the reference directory.")

In [ ]:
title_list = []
source_list = []
ref_itd_list = []
est_itd_list = []
ref_ild_list = []
est_ild_list = []

print("Beginning to evaluate stems...")
for source in STEMS:
    print(f"\n>>>>{source} <<<<")
    for song in tqdm(song_list):
        
        ref_file = os.path.join(REFERENCE_DIR, song, f"{source}.wav")
        est_file = os.path.join(ESTIMATE_DIR, song, f"{source}.wav")

        # load reference and estimate stems
        y_ref, sr_ref = sf.read(ref_file)
        y_est, sr_est = sf.read(est_file)

        # check sample rates
        assert sr_ref == sr_est

        # calculate ITD
        itd_ref = compute_itd(y_ref, sr_ref)
        itd_est = compute_itd(y_est, sr_est)

        # calculate ILD
        ild_ref = compute_ild(y_ref)
        ild_est = compute_ild(y_est)

        title_list.append(song)
        source_list.append(source)
        ref_itd_list.append(itd_ref)
        est_itd_list.append(itd_est)
        ref_ild_list.append(ild_ref)
        est_ild_list.append(ild_est)

results_df = pd.DataFrame({"title": title_list, "source": source_list,
                           "ref_ITD": ref_itd_list, "est_ITD": est_itd_list,
                           "ref_ILD": ref_ild_list, "est_ILD": est_ild_list})

print("Evaluation complete!")

In [ ]:
# sort
results_df.sort_values(by=['title', 'source'], inplace=True, ignore_index=True)

# computer delta itd
results_df['diff_ITD'] = (results_df['ref_ITD'] - results_df['est_ITD']).abs()

# computer delta ild
results_df['diff_ILD'] = (results_df['ref_ILD'] - results_df['est_ILD']).abs()

In [ ]:
save_path = os.path.join(EVAL_DIR, f'itd_ild_{DATE}_{MODEL}_{DATASET}.csv')

results_df.to_csv(save_path, index=False)

### Standard Binaural Data

In [ ]:
DATASET = 'standard'

# set input and output directories
REFERENCE_DIR = f"../data/binaural_musdb18/{DATASET}/test/"
ESTIMATE_DIR = f"../data/output/{MODEL}/{DATASET}/test/"

In [ ]:
# create the output directory if it does not already exist
print("Creating evaluation directory, if it does not already exist...")
os.makedirs(EVAL_DIR, exist_ok=True)

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
song_list = [f for f in os.listdir(REFERENCE_DIR) if os.path.isdir(os.path.join(REFERENCE_DIR, f))]
print(f"There are {len(song_list)} files in the reference directory.")

In [ ]:
title_list = []
source_list = []
ref_itd_list = []
est_itd_list = []
ref_ild_list = []
est_ild_list = []

print("Beginning to evaluate stems...")
for source in STEMS:
    print(f"\n>>>>{source} <<<<")
    for song in tqdm(song_list):
        
        ref_file = os.path.join(REFERENCE_DIR, song, f"{source}.wav")
        est_file = os.path.join(ESTIMATE_DIR, song, f"{source}.wav")

        # load reference and estimate stems
        y_ref, sr_ref = sf.read(ref_file)
        y_est, sr_est = sf.read(est_file)

        # check sample rates
        assert sr_ref == sr_est

        # calculate ITD
        itd_ref = compute_itd(y_ref, sr_ref)
        itd_est = compute_itd(y_est, sr_est)

        # calculate ILD
        ild_ref = compute_ild(y_ref)
        ild_est = compute_ild(y_est)

        title_list.append(song)
        source_list.append(source)
        ref_itd_list.append(itd_ref)
        est_itd_list.append(itd_est)
        ref_ild_list.append(ild_ref)
        est_ild_list.append(ild_est)

results_df = pd.DataFrame({"title": title_list, "source": source_list,
                           "ref_ITD": ref_itd_list, "est_ITD": est_itd_list,
                           "ref_ILD": ref_ild_list, "est_ILD": est_ild_list})

print("Evaluation complete!")

In [ ]:
# sort
results_df.sort_values(by=['title', 'source'], inplace=True, ignore_index=True)

# computer delta itd
results_df['diff_ITD'] = (results_df['ref_ITD'] - results_df['est_ITD']).abs()

# computer delta ild
results_df['diff_ILD'] = (results_df['ref_ILD'] - results_df['est_ILD']).abs()

In [ ]:
save_path = os.path.join(EVAL_DIR, f'itd_ild_{DATE}_{MODEL}_{DATASET}.csv')

results_df.to_csv(save_path, index=False)

### Random Binaural Data

In [ ]:
DATASET = 'random'

# set input and output directories
REFERENCE_DIR = f"../data/binaural_musdb18/{DATASET}/test/"
ESTIMATE_DIR = f"../data/output/{MODEL}/{DATASET}/test/"

In [ ]:
# create the output directory if it does not already exist
print("Creating evaluation directory, if it does not already exist...")
os.makedirs(EVAL_DIR, exist_ok=True)

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
song_list = [f for f in os.listdir(REFERENCE_DIR) if os.path.isdir(os.path.join(REFERENCE_DIR, f))]
print(f"There are {len(song_list)} files in the reference directory.")

In [ ]:
title_list = []
source_list = []
ref_itd_list = []
est_itd_list = []
ref_ild_list = []
est_ild_list = []

print("Beginning to evaluate stems...")
for source in STEMS:
    print(f"\n>>>>{source} <<<<")
    for song in tqdm(song_list):
        
        ref_file = os.path.join(REFERENCE_DIR, song, f"{source}.wav")
        est_file = os.path.join(ESTIMATE_DIR, song, f"{source}.wav")

        # load reference and estimate stems
        y_ref, sr_ref = sf.read(ref_file)
        y_est, sr_est = sf.read(est_file)

        # check sample rates
        assert sr_ref == sr_est

        # calculate ITD
        itd_ref = compute_itd(y_ref, sr_ref)
        itd_est = compute_itd(y_est, sr_est)

        # calculate ILD
        ild_ref = compute_ild(y_ref)
        ild_est = compute_ild(y_est)

        title_list.append(song)
        source_list.append(source)
        ref_itd_list.append(itd_ref)
        est_itd_list.append(itd_est)
        ref_ild_list.append(ild_ref)
        est_ild_list.append(ild_est)

results_df = pd.DataFrame({"title": title_list, "source": source_list,
                           "ref_ITD": ref_itd_list, "est_ITD": est_itd_list,
                           "ref_ILD": ref_ild_list, "est_ILD": est_ild_list})

print("Evaluation complete!")

In [ ]:
# sort
results_df.sort_values(by=['title', 'source'], inplace=True, ignore_index=True)

# computer delta itd
results_df['diff_ITD'] = (results_df['ref_ITD'] - results_df['est_ITD']).abs()

# computer delta ild
results_df['diff_ILD'] = (results_df['ref_ILD'] - results_df['est_ILD']).abs()

In [ ]:
save_path = os.path.join(EVAL_DIR, f'itd_ild_{DATE}_{MODEL}_{DATASET}.csv')

results_df.to_csv(save_path, index=False)

## Appendix

In [ ]:
# azi_angles = np.arange(0, 360, 5)

# itd_list = []

# for theta in azi_angles:
#     hrir, sr = sf.read(f'../data/D1_HRIR_WAV/44K_16bit/azi_{theta},0_ele_0,0.wav')
#     itd_list.append(compute_itd(hrir, sr))

# plt.plot(azi_angles, itd_list)
# plt.ticklabel_format(axis='y', style='sci',  scilimits=(-3,-3))
# plt.xlabel("$\Theta$ Azimuth")
# plt.ylabel("ITD (s)")
# plt.ylim([-1e-3, 1e-3])
# plt.show()